In [6]:
import numpy as np
from aifunc import *

In [7]:
# momentum
# v = a*v - lr * dL/dw
# W = W + v  ... v는 누적  v초기값은 0, a는 0.9정도..
# v는 누적이면서 점점 0.9씩 작이짐.
# 마치 물리 법칙처럼 가속하듯이 누적 but 0.9를 곱하면서 가속도 점점 줄어듬.
# 지그제그로 움직이지 않고 부드럽게 움직인다.

class Momentum:

    """모멘텀 SGD"""

    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
        # 매개변수에 v를 더하면서 나아간다
        # v도 기울기를 빼주며 값이 누적된다.
        # 원래 값에 0.9씩 곱하며 
        # 마치 물리 법칙처럼 가속하듯이 누적 but 0.9를 곱하면서 가속도 점점 줄어듬.
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]

In [8]:
# 각각의 매개변수 별로 학습률을 조정해 주겠다~~
# 학습 더 할 필요 없는 애들은 lr을 낮추고 더 필요한 애는 더 키운다.

# h = h + dL/dW  *d/L*d/W  (기울기 원소별 제곱)!
# W = W - lr * 1/squ(h) * dL/dW

# h에는 계속 기울기의 제곱을 누적
# 누적값이 많아질 수록 lr을 낮추는 방식!
# h가 너무 작아져 0에 가까워지면 학습이 일어나지 않을 수 있다...
# 누적중에 오래된 weight 기울기는 잊어버리게 하면, 문제를 조금 해결할 수 있다. 
class AdaGrad:

    """AdaGrad"""
    # 각각 매개변수마다 학습률의 크기가 맞춰진다.

    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        # h라는 변수 추가
        # h는 기울기의 제곱의 누적합
        # 실제 W는 learning rate / h의 1/2승   즉..h가 클수록 learning rate가 작아진다.
        # 각 매백변수별로 lr이 맞춰진다.. 기울기가 크게 움직일 수록 더 세밀하게 움직이게끔 조정
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

In [9]:
class Adam:
    ##
    # AdaGrad와 moment의 융합
    # 점점 스스로를 가속하면서
    # lr을 세밀하게 조정해나간다.
    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            # m -> moment 부분
            # v -> AdaGrad 부분
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
